## FHIR Implementation Guide Testing Pipeline
This notebook provides a comprehensive pipeline for automatically extracting requirements from FHIR Implementation Guides and generating executable test suites. The pipeline transforms Implementation Guide (IG) documentation into structured test code that can validate FHIR server implementations.

#### Overview
This automated pipeline takes FHIR Implementation Guide documentation and produces comprehensive test suites through several integrated stages:

- Implementation Guide Preparation: Convert and clean IG HTML documentation to markdown format
- Requirements Extraction: Use AI to identify and extract testable requirements from the IG
- Requirements Refinement: Consolidate and refine the extracted requirements
- Requirements Downselection: Combine multiple requirement sets and remove duplicates
- Test Plan Generation: Convert requirements into detailed test specifications
- Test Kit Generation: Generate executable Inferno test code

#### Running this Notebook
The notebook is structured to run each stage sequentially. You can either:

- Run the complete pipeline: Execute all cells to process a complete IG
- Run individual stages: Execute specific sections as needed

Inputs and output directories can be customized for each step. The pipeline automatically saves intermediate outputs in checkpoint directories for review and iteration.

#### Output Structure
The pipeline generates organized outputs in checkpoint directories:

checkpoints/

├── markdown1/          # Converted markdown files

├── markdown2/          # Cleaned markdown files  

├── requirements_extraction/   # Initial AI-extracted requirements

├── revised_reqs_extraction/  # Refined requirements lists

├── requirements_downselect/  # Final consolidated requirements

├── testplan_generation/     # Detailed test specifications

└── testkit_generation/      # Executable Inferno test suites

Each stage preserves its outputs, allowing for iteration, review, and alternative processing paths.

## Setup

#### Importing required packages

In [ ]:
import inspect
import json
import llm_utils
import importlib
import tiktoken
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from glob import glob

## Initializing LLM Clients

In [ ]:
importlib.reload(llm_utils)
llm_clients = llm_utils.LLMApiClient()

In [ ]:
llm_clients.clients

## Implementation Guide Preparation

### Stage 1: Text Extraction and Cleaning
- Converts HTML IG files to markdown format
- Cleans unnecessary content (navigation, headers, formatting artifacts)
- Prepares clean, structured text for AI processing

Inputs: HTML files from FHIR IG downloads

Outputs: Clean markdown files

#### 1a) HTML to Markdown Conversion

In [ ]:
import html_narrative_extractor_01 #import html extractor module

# Process directory with default settings
result = html_narrative_extractor_01.convert_local_html_to_markdown(
    input_dir="../us-core/test_set", #input directory of downloaded IG HTML files
    output_dir="checkpoints/demo/markdown1/" #output directory
)

#### 1b) Markdown Post-processing

In [ ]:
import markdown_cleaner_02 #import markdown cleaner module
markdown_cleaner_02.process_directory(
    input_dir="checkpoints/demo/markdown1", #input directory of IG markdown files
    output_dir="checkpoints/demo/markdown2/") #output directory

## Stage 2: Requirements Extraction

### 2a) Prompt-based Requirement Extraction
LLM Requirements Identification
- Processes markdown files using LLM to extract clear, testable requirements
- Formats requirements according to set standards, following INCOSE guidance
- Generates structured requirements with IDs, descriptions, actors, and conformance levels
- Handles large documents through chunking
- Provides source tracking

Inputs: Cleaned IG markdown files

Outputs: Structured requirements list as markdown file

In [ ]:
import reqs_extraction_03 #import LLM requirements extraction module
importlib.reload(reqs_extraction_03)

In [ ]:
reqs_extraction_03.run_requirements_extractor(
    markdown_dir='checkpoints/demo/condition_profile', #input directory of markdown files
    output_dir='checkpoints/demo/requirements_extraction/', #output directory
    api_type= 'claude', #set API type
    client_instance=llm_clients) #initialize llm clients

### 2b) Requirements Refinement
LLM-Based Requirements Review & Consolidation
- Filters and identifies only testable requirements from raw extractions
- Consolidates duplicate requirements and merges related ones
- Applies consistent formatting and structure
- Removes non-testable assertions and narrative content

Inputs: Raw requirements from extraction stage in markdown format

Outputs: Refined requirements list in markdown format

In [ ]:
# import requirements refinement script as module
import reqs_reviewer_04
importlib.reload(reqs_reviewer_04)

In [ ]:
result = reqs_reviewer_04.run_batch_requirements_refinement(
    input_file="checkpoints/demo/requirements_extraction/reqs_list_v1.md", #input requirements list markdown file
    output_dir="checkpoints/demo/requirements_revision/", #output directory   
    client_instance=llm_clients,  #initialize llm clients
    batch_size=25,  #set batch size
    api_type="claude"  #set API type
)

### 2c) Requirements Downselection
- Combines multiple requirements lists from different extraction runs
- Uses semantic similarity analysis to identify and remove duplicates
- Creates a deduplicated final requirements set

Inputs: Multiple refined requirements files in markdown or JSON format
Outputs: Final consolidated requirements in markdown or JSON format

In [ ]:
import reqs_downselect_05
importlib.reload(reqs_downselect_05)

In [ ]:
md_files_list=reqs_downselect_05.get_md_files_from_directory("checkpoints/demo/requirements_revision/")

reqs_downselect_05.full_pass(
    md_files=md_files_list,
    output_dir="checkpoints/demo/requirements_downselect"
)

## Stage 3: Test Plan Generation
- Transforms requirements into detailed test specifications
- Analyzes IG capability statements for context
- Generates implementation strategies with specific FHIR operations
- Creates structured test plans with validation criteria

Inputs: Refined requirements and IG capability statements in markdown format

Outputs: Detailed test plan in markdown format

In [ ]:
import logging
llm_clients.logger.setLevel(logging.INFO)

In [ ]:
import warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set logging level to reduce noise
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
logging.getLogger("backoff").setLevel(logging.ERROR)

import test_plan_06 #import test plan generation script as module
importlib.reload(test_plan_06)

#clearing any existing capability statements from vector database
test_plan_06.clear_capability_collection("capability_statements")

In [ ]:
test_plan_06.generate_consolidated_test_plan(
    client_instance=llm_clients, 
    api_type='claude',
    requirements_file="checkpoints/demo/requirements_downselect/consolidated_reqs.md", #input requirements list markdown file
    capability_statement_file="checkpoints/demo/markdown2/CapabilityStatement-us-core-server.md", 
    ig_name="US Core IG", 
    output_dir='checkpoints/demo/testplan_generation/', 
    verbose=True)

## Stage 4: Test Kit Generation
- Converts test specifications into executable Inferno Ruby tests
- Generates complete test suites with proper file organization
- Creates modular test structures following Inferno framework patterns
- Includes validation and alignment checking

Inputs: Test plan specification in markdown format

Outputs: Complete Inferno test kit

In [ ]:
import test_kit_07
importlib.reload(test_kit_07)

Without LLM Self Evaluation

In [ ]:
# Faster generation- no LLM self evaluation
test_kit_07.generate_inferno_test_kit(
    client_instance=llm_clients, #initialize llm clients
    api_type='claude',  #set API
    test_plan_file='checkpoints/demo/testplan_generation/test_plan.md',  #input test plan file
    ig_name='US Core',
    output_dir='checkpoints/demo/testkit_generation/',
    enable_validation=False  #disable LLM self evaluation
)

With LLM Self Evaluation

In [ ]:
# Thorough generation- with LLM self evaluation
test_kit_07.generate_inferno_test_kit(
    client_instance=llm_clients, #initialize llm clients
    api_type='claude',  #set API
    test_plan_file='checkpoints/demo/testplan_generation/test_plan.md',  #input test plan file
    ig_name='US Core',
    output_dir='checkpoints/demo/testkit_generation/',
    enable_validation=True  #enable LLM self evaluation
)